In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai import *
from fastai.text import *

# Wikitext 103

In [3]:
PATH = Path('../data/wikitext-103/60k')

In [4]:
def read_file(filename):
    with open(PATH/f'{filename}.txt', encoding='utf8') as f:
        text = f.readlines()
    df = pd.DataFrame({'text':np.array(text), 'labels':np.zeros(len(text))}, columns=['labels', 'text'])
    df.to_csv(PATH/f'{filename}.csv', header=False, index=False)

In [9]:
read_file('valid')
#read_file('test')
read_file('train')

In [10]:
tokenizer = Tokenizer(rules=rules, special_cases=[BOS, FLD, 'xxunk', 'xxpad'])
train_ds, valid_ds = TextDataset.from_csv(PATH, tokenizer, max_vocab=60000)

Tokenizing train. This might take a while so you should grab a coffee.


Numericalizing train.
Tokenizing valid. This might take a while so you should grab a coffee.


Numericalizing valid.


In [11]:
bs,bptt = 80,70
train_dl = LanguageModelLoader(np.concatenate(train_ds.ids), bs, bptt)
valid_dl = LanguageModelLoader(np.concatenate(valid_ds.ids), bs, bptt)

In [12]:
vocab_size = len(train_ds.vocab.itos)
data = DataBunch(train_dl, valid_dl)

In [13]:
emb_sz, nh, nl = 400, 1150, 3
dps = np.array([0.25, 0.1, 0.2, 0.02, 0.15]) * 0.1
model = get_language_model(vocab_size, emb_sz, nh, nl, 0, input_p=dps[0], output_p=dps[1], weight_p=dps[2], 
                           embed_p=dps[3], hidden_p=dps[4])
learn = Learner(data, model)

In [14]:
learn.opt_fn = partial(optim.Adam, betas=(0.8,0.99))
learn.callbacks.append(RNNTrainer(learn, bptt, alpha=2, beta=1))
learn.callback_fns.append(partial(GradientClipping, clip=0.12))
learn.metrics = [accuracy]
learn.true_wd=False

In [15]:
fit_one_cycle(learn, 10, 5e-3, (0.8,0.7), wd=1e-7)

Total time: 10:28:58
epoch  train loss  valid loss  accuracy
0      3.867067    3.917060    0.357719  (1:03:08)
1      3.736901    3.824207    0.363141  (1:03:02)
2      3.734958    3.847070    0.360801  (1:02:49)
3      3.690017    3.810396    0.364572  (1:02:50)
4      3.615047    3.704101    0.374547  (1:02:58)
5      3.555377    3.589375    0.387541  (1:02:58)
6      3.526703    3.495668    0.399273  (1:02:42)
7      3.531244    3.480701    0.401607  (1:02:47)
8      3.505409    3.472463    0.402647  (1:02:46)
9      3.501862    3.468110    0.403324  (1:02:53)



In [16]:
learn.save('lstm_wt103')